In [1]:
#1-ci
import collections
import os
import json
import nltk
import csv
import pandas as pd
def load_all_jsonl(path = os.getcwd()):
    data = []
    for file_name in os.listdir(path):
        if "jsonl" in file_name:
            if file_name.split('.')[1] == 'jsonl':
                print(file_name)
                with open(file_name, 'r', encoding='utf-8') as reader:
                    for line in reader:
                        data.append(json.loads(line))
    return data    
data = load_all_jsonl()

erkinaz_m_wikipedia.jsonl
esmira_m_wikipedia.jsonl
ilahe_m_wikipedia.jsonl
nubar_m_wikipedia.jsonl
sevinc_m_wikipedia.jsonl


In [2]:
data

[{'id': 45001,
  'text': 'İnsanlarla dil tapmağı bacarırdı',
  'label': [[0, 8, 'İsim'],
   [8, 10, 'Qoş-Vasitə_birgə'],
   [11, 22, 'Feil'],
   [23, 32, 'Feil']]},
 {'id': 45002,
  'text': 'Prospekt Heydər Əliyev Sarayından Dördyol adlanan meydana qədər uzanır',
  'label': [[0, 8, 'İsim'],
   [9, 33, 'İsim-Mürəkkəb_Ad'],
   [34, 41, 'İsim-Xüsusi'],
   [42, 49, 'Feil-Sifət'],
   [50, 57, 'İsim'],
   [58, 63, 'Qoş-Məsafə'],
   [64, 70, 'Feil']]},
 {'id': 45003,
  'text': 'Bu film də ona böyük uğur gətirmişdir',
  'label': [[0, 2, 'Əvəzlik'],
   [3, 7, 'İsim'],
   [8, 10, 'Bağ-İştirak'],
   [11, 14, 'Əvəzlik-Şəxs'],
   [15, 20, 'Sifət'],
   [21, 25, 'İsim'],
   [26, 37, 'Feil']]},
 {'id': 45004,
  'text': 'Postament dəmir-beton konstruksiyasından ibarətdir və mərmər plitələrlə üzlənmişdir',
  'label': [[0, 9, 'İsim'],
   [10, 21, 'İsim'],
   [22, 40, 'İsim'],
   [41, 50, 'Köməkçi_söz'],
   [51, 53, 'Bağ-Birləşdirmə'],
   [54, 60, 'İsim'],
   [61, 69, 'İsim'],
   [69, 71, 'Qoş-Vasitə_birg

In [ ]:
data

In [3]:
#2-ci
#data label olunmayan cumleleri lst listine yigir sonra data-dan silinecek

lst=[]
for i in range(len(data)):
    if len(data[i]['entities'])==0:        
        lst.append(i)
        
for i in lst:
    data.remove(data[i])
len(data)

901

In [5]:
#3-cu
# birinci hisse start indekse gore ardicil duzur

l1,l2,l3,l4=[],[],[],[]
for i in range(len(data)):
    for j in data[i]['entities']:
        l1.append(j['start_offset'])
    l2=l1.copy()
    l2.sort()
    for k in l2:    
        l3.append(data[i]['entities'][l1.index(k)])
    l4.append(l3)
    l1,l2,l3=[],[],[]

# 2-ci hisse tekrar ve murekkeb isimle pararlel label olmush isimleri silir
x,y=[],[]
temporary_list1,temporary_list2=[],[]
for i in l4:
    for j in i:
        if j not in temporary_list1:
            temporary_list1.append(j)
    temporary_list2.append(temporary_list1)
    temporary_list1=[]
    
for i in temporary_list2:
    for j in range(len(i)):
        if j==0:
            x.append(i[j])
        else:
            if (i[j]['start_offset']>i[j-1]['start_offset'] and i[j]['end_offset']>i[j-1]['end_offset']):
                x.append(i[j])
    y.append(x)
    x=[]  

# ardicil duzulmush ve tekrar labeller ve murekkeb isim sherti odenilmish label dictionarylerin data entities-e kocurulmesi

for i in range(len(data)):
    data[i]['entities']=y[i]

#label olunmayan cumlelerin data-dan silinmesi
for i in data:
    if len(i['entities'])==0:
        data.remove(i)
        
#textden '---' evvel yazilmish hisseni silir ve indeksleri yenileyir
l1,l2,l3,l4,lst2=[],[],[],[],[]
for i in data:
    try:
        x=i['entities'][0]['start_offset']
        l1.append(i['text'][i['entities'][0]['start_offset']:].replace('\n',' '))
        for j in i['entities']:
            l2.append(j['label'])
            l2.append(j['start_offset']-x)
            l2.append(j['end_offset']-x)
            l3.append(l2)
            l2=[]
        l4.append(l3)
        l3=[]
    except:
        lst2.append(i)
#entities lsitleri bosh olan datanin silinmesi, label olayan cumlelerin silinmesi

#evvelki sozun sonu sonraki sozun bashlangicindan boyuk olarsa duzelish edir
for i in range(len(l4)):
    for j in range(len(l4[i])-1):
        if l4[i][j][2]>l4[i][j+1][1]:
            l4[i][j][2]=l4[i][j][2]-1 
            
# #evvelki sozun sonu sonraki sozun bashlangici ile eyni olarsa duzelish edir
for i in range(len(l4)):
    for j in range(len(l4[i])-1):
        if l4[i][j][2]==l4[i][j+1][1] and l4[i][j+1][0]!='Cümlə_sonu' and len(l1[i][l4[i][j][1]:l4[i][j+1][2]].split())==2:
            l4[i][j][2]= l4[i][j][2] - 1

In [5]:
#4-cu
#datadan label olunmayan melumatin silinmesi novbeti kod setri ishleyir exceptleri lst2 listine yazir onu burda istifade edirik
for i in lst2:
    data.remove(i)

In [7]:
#5
#bu kod ishe salinir alinan neticeler sehv olanda onlari goturub evvelde silirik
# n=0
# for i in range(len(l1)):
#     for j in range(len(l4[i])-1):
#         if l4[i][j][2]==l4[i][j+1][1] and l4[i][j+1][0]!='Cümlə_sonu' and len(l1[i][l4[i][j][1]:l4[i][j+1][2]].split())==1:
            
#             n+=1
#             print(i) 
#             print(l4[i][j]) 
#             print(l4[i][j+1]) 
#             print(l1[i]) 
#             print(l1[i]) 
#             print(j) 
#             print(l1[i][l4[i][j][1]:l4[i][j][2]])
#             print(l1[i][l4[i][j+1][1]:l4[i][j+1][2]])
#             print()
# print(n)

In [6]:
#6
#sehv label olunmush say ve noqte ve s. silinmesi #5-in cavablari burda lsite qoyulur.
ll=[454,645,1210,1421,1440,1632,2106,2530,2575,2577,2586,2608,2643,2668,2691,
   2693,2714,2848,2921,2947,2986,2987,2993,2995,3017,3033,3127,3133,3330,3365,
   3409,3461,3504,3722,3723,3905,4111,4170,4212,4280,4385,4472,4496,4498,4650,
   4655,4690,4720,4747,4805,4938,4943,5169,6041,6045,6315,6325,6344,6373,6442,
   6556,6559,7054,7097,7143,7418,8234,9317,9341,9973,10013,10318,10751,10986,
   10996,11046,11340,11698,11774,11828,]

for i in sorted(ll, reverse=True):
    del l1[i]
    del l4[i]

In [7]:
#7
#Is ve le olan sozlerin labellerini birlshdirir
lll=[]
for i in range(len(l4)):
    for j in range(len(l4[i])-1):
        
        if l4[i][j][2]==l4[i][j+1][1] and l4[i][j+1][0]!='Cümlə_sonu' and len(l1[i][l4[i][j][1]:l4[i][j+1][2]].split())==1:
            
            lll.append(l4[i][j][0]+'/'+l4[i][j+1][0])
            lll.append(l4[i][j][1])
            lll.append(l4[i][j+1][2])
            l4[i][j]=lll
            lll=[]
            l4[i][j+1]=['sil',100000,1000000]
            
for i in l4:
    for j in i:
        if j[0]=='sil' and j[1]==100000:
            i.remove(j)

In [8]:
#8 
# data ilk defe olanda 9-ni run edib bura yaziriq eceptionlari sonraki defelerde 8 sonra 7 run edilir.

l4[2636][14]=['Say', 111, 112]
l4[2686][12]=['Say', 80, 81]
l4[4897][9]=['Cümlə_sonu', 103, 104]

In [9]:
#9
#indekslerin duzgun yerleshdirilmesi

for j in range(len(l1)):
    #print(l1[j])
    s=l1[j]
    
    for i in l4[j]:
        #print(i)   
        # 'men '----> [0:1] cevirir [0:3] edir
        if i[0] == "Cümlə_sonu":
            continue
        if i[1]==0:

            # b qabaga
            if not s[i[1]].isalnum():
                while not s[i[1]].isalnum():
                    i[1]=i[1]+1

            # s arxaya
            if i[2] != 1 and not s[i[2]-1].isalnum():
                while not s[i[2]-1].isalnum():
                    i[2]=i[2]-1
            else:
                # s qabaga
                if i[2] !=len(s) and i[2] < len(s) and s[i[2]].isalnum():
                    while i[2] !=len(s) and s[i[2]].isalnum():
                        i[2]=i[2]+1
                    
        else:
            # b qabaga
            try:
                if not s[i[1]].isalnum():
                    while not s[i[1]].isalnum(): #Nurlan
                        i[1]=i[1]+1
            except IndexError:
                print(l1[j])
                print(j)
                print(i)
                print()


            # b arxaya
            else:
                if s[i[1]-1].isalnum():
                    while s[i[1]-1].isalnum():
                        i[1]=i[1]-1
            
            # s arxaya
            if i[2] != 1 and not s[i[2]-1].isalnum():
                while not s[i[2]-1].isalnum():
                    i[2]=i[2]-1

            else:
                # s qabaga
                if i[2] !=len(s) and i[2] < len(s) and s[i[2]].isalnum():
                    while i[2] !=len(s) and s[i[2]].isalnum():
                        i[2]=i[2]+1

In [10]:
#10
#eyni soz mextelif label-le labellenibse, onlari birlehsdirir: Feil-inkar/Feil_Sifet
lll=[]
for i in range(len(l4)):
    for j in range(len(l4[i])-1):
        
        if l4[i][j][1]==l4[i][j+1][1] and l4[i][j][2]==l4[i][j+1][2] and l4[i][j][0]!=l4[i][j+1][0]:
           
            lll.append(l4[i][j][0]+'/'+l4[i][j+1][0])
            lll.append(l4[i][j][1])
            lll.append(l4[i][j][2])
            l4[i][j]=lll
            lll=[]
            l4[i][j+1]=['sil',100000,1000000]
            
for i in l4:
    for j in i:
        if j[0]=='sil' and j[1]==100000:
            i.remove(j)       

In [11]:
#11
#BME uzre bolunme hissesi
y,z=[],[]

for i in range(len(l1)):
    for j in (l4[i]):        
        x=l1[i][j[1]:j[2]].split()
        
        if len(x)>=2:
            
            n=(j[1]+len(x[0])+1)
            
            for ii in range(len(x)):
                
                if ii==0:
                    y.append("B/"+j[0])
                    y.append(j[1])
                    y.append(j[1]+len(x[0]))
                    l4[i].append(y)
                    
                elif ii==len(x)-1:
                    y.append("E/"+j[0])
                    y.append(n)
                    y.append(n+len(x[ii]))
                    l4[i].append(y)

                else:                                   
                    y.append("M/"+j[0])
                    y.append(n)
                    y.append(n+len(x[ii]))
                    l4[i].append(y)
                    y=[]
                    n+=(len(x[ii])+1)
                y=[]
            n=0
            j=['silhokmen',0,1]
for i in l4:
    for j in i:
        if j[0]=='silhokmen':
            i.remove(j)
        
# l4 listinde olan itemlerin ardicil duzulmesi; BME duzelishi eden zaman yeni B,M,E her biri listin sonuna append olundugundan
# ardicilliq pozulub. Yeni indeksler l5 listinde, cumleler l1 listinde yigilib
list_needed,l5 = [],[]
dict_ = {}
index = 0

for a in l4:
    for list_ in a:
        dict_[list_[1]] = index
        index += 1
        
    for i in collections.OrderedDict(sorted(dict_.items())).values():
        list_needed.append(a[i])
    l5.append(list_needed)
    list_needed=[]
    dict_={}
    index=0

In [12]:
#12
#M/ olan ve reqem ve herf olmayanlarin silinmesi
for i in range(len(l1)):
    for j in range(len(l5[i])):
        if l5[i][j][0][0:2]=='M/' and not l1[i][l5[i][j][1]:l5[i][j][2]].isalnum():
            l5[i][j]=['sil2',100000000,100000000]  
for i in l5:
    for j in i:
        if j[0]=='sil2' and j[1]==100000000 and j[2]==100000000:
            i.remove(j)

In [25]:
#l5[314].remove(l5[314][10])
#l5[3108].remove(l5[3108][10])
# l5[5794].remove(l5[3108][10])

In [13]:
#13
for i in range(len(l5)):
    for j in l5[i]:
        if j[0]=='sil2':
            l5[i].remove(j)

In [14]:
#14
#indekslerin duzgun yerleshdirilmesi

for j in range(len(l1)):
    #print(l1[j])
    s=l1[j]
    
    for i in l5[j]:
        #print(i)   
        # 'men '----> [0:1] cevirir [0:3] edir
        if i[0] == "Cümlə_sonu":
            continue
        if i[1]==0:

            # b qabaga
            if not s[i[1]].isalnum():
                while not s[i[1]].isalnum():
                    i[1]=i[1]+1

            # s arxaya
            if i[2] != 1 and not s[i[2]-1].isalnum():
                while not s[i[2]-1].isalnum():
                    i[2]=i[2]-1
            else:
                # s qabaga
                if i[2] !=len(s) and i[2] < len(s) and s[i[2]].isalnum():
                    while i[2] !=len(s) and s[i[2]].isalnum():
                        i[2]=i[2]+1
                    
        else:
            # b qabaga
            try:
                if not s[i[1]].isalnum():
                    while not s[i[1]].isalnum(): #Nurlan
                        i[1]=i[1]+1
            except IndexError:
                print(l1[j])
                print(j)
                print(i)
                print()


            # b arxaya
            else:
                if s[i[1]-1].isalnum():
                    while s[i[1]-1].isalnum():
                        i[1]=i[1]-1
            
            # s arxaya
            if i[2] != 1 and not s[i[2]-1].isalnum():
                while not s[i[2]-1].isalnum():
                    i[2]=i[2]-1

            else:
                # s qabaga
                if i[2] !=len(s) and i[2] < len(s) and s[i[2]].isalnum():
                    while i[2] !=len(s) and s[i[2]].isalnum():
                        i[2]=i[2]+1

In [15]:
#15
#14-un cavablari burda run edilir duzelish ucun
l5[3121][11]=['Say', 74, 78]
l5[5471][7]=['İsim', 73, 78]

In [17]:
#16
#l5-de olan labellerin ve l1-de olan cumlelerin txt-lere yazilmasi

string_=''
label_=''

f1 = open("Cumleler36.txt", "w", encoding="utf-8")
f2 = open("label36.txt", "w", encoding="utf-8")

for i in range(len(l1)):
    
    for j in l5[i]:
        if j[0]!='Cümlə_sonu':
            string_+=l1[i][j[1]:j[2]]+' '
            label_+=j[0]+' '
        
    f1.write(string_.strip()) 
    f1.write("\n")
    
    f2.write(label_.strip()) 
    f2.write("\n")
    string_=''
    label_=''
    
f1.close()
f2.close()

In [18]:
#17
f1  = open("Cumleler36.txt", "r", encoding="utf-8")
f2  = open("label36.txt", "r", encoding="utf-8")
c1 = f1.readlines()
c2 = f2.readlines()

for i in range(len(c1)):
    if len(c1[i].split())!=len(c2[i].split()):
        print(i)
                
f1.close()
f2.close()

In [1]:
#datanin qarishdirilmasi

import random
lst=[]

f1  = open("Cumleler136.txt", "r", encoding="utf-8")
f2  = open("labeller136.txt", "r", encoding="utf-8")
c1 = f1.readlines()
c2 = f2.readlines()

f11 = open("Cumleler_qarishiq136.txt", "w", encoding="utf-8")
f22 = open("labeller_qarishiq136.txt", "w", encoding="utf-8")

while len(lst)<len(c1):
    x=random.randint(0,len(c1))
    if x not in lst:
        lst.append(x)
        
for i in lst:
    try:
        f11.write(c1[i]) 
        #f11.write("\n")

        f22.write(c2[i]) 
        #f22.write("\n")
    except:
        print(i)
f1.close()
f2.close()
f11.close()
f22.close()

30689


In [20]:
print(len(c1))

11845


In [1]:
# l2=['la','lə','ca','cə','mı']
# l3=['dək','tək','can','cən']
# l4=['sana']

In [59]:
# for i in l5:
#     for j in i:
#         if j[0][0:2] not in ['B/','M/','E/'] and j[0].find('/')>0:
            
#             print(j[0])

In [60]:
#BME-leri yeniden birleshdirir
# l7=[]
# d={}
# s5=[
#   {
#     "label": "B/İsim-Mürəkkəb_Ad",
#     "start_offset": 0,
#     "end_offset": 6
#   },
#   {
#     "label": "M/İsim-Mürəkkəb_Ad",
#     "start_offset": 11,
#     "end_offset": 16
#   },
#   {
#     "label": "E/İsim-Mürəkkəb_Ad",
#     "start_offset": 17,
#     "end_offset": 23
#   }
# ]

# for i in s5:
#     if i['label'].split('/')[0] not in ['B','M','E']:
#         l7.append(i)
#     else:      
#         if i['label'].split('/')[0]=='B':
            
#             d['label']=i['label'].split('/')[1]
#             d['start_offset']=i['start_offset']
            
#         elif i['label'].split('/')[0]=='E':
#             d['end_offset']=i['end_offset']
#             l7.append(d)
#             d={}
# l7

In [54]:
# is ve le birleshmesini ayiran proqram
# l14=[]

# s7='Bəzən bir-birimizə öz arzularımızı deyir, müharibədən sonra nə işlə məşğul olacağımızın xəyalını qururduq.'
# l12=[
#   {
#     "label": "Zərf",
#     "start_offset": 0,
#     "end_offset": 5
#   },
#   {
#     "label": "Əvəzlik",
#     "start_offset": 6,
#     "end_offset": 18
#   },
#   {
#     "label": "Əvəzlik",
#     "start_offset": 19,
#     "end_offset": 21
#   },
#   {
#     "label": "İsim",
#     "start_offset": 22,
#     "end_offset": 34
#   },
#   {
#     "label": "Feil-Təsirli",
#     "start_offset": 35,
#     "end_offset": 40
#   },
#   {
#     "label": "İsim",
#     "start_offset": 42,
#     "end_offset": 53
#   },
#   {
#     "label": "Qoş-Zaman",
#     "start_offset": 54,
#     "end_offset": 59
#   },
#   {
#     "label": "Əvəzlik",
#     "start_offset": 60,
#     "end_offset": 62
#   },
#   {
#     "label": "İsim/Qoş-Vasitə_birgə",
#     "start_offset": 63,
#     "end_offset": 67
#   },
#   {
#     "label": "Feil",
#     "start_offset": 68,
#     "end_offset": 87
#   },
#   {
#     "label": "Feil",
#     "start_offset": 88,
#     "end_offset": 105
#   }
# ]

# for i in l12:
#     if '/' in i['label']:
        
#         if s7[i["start_offset"]:i["end_offset"]][-3:] in ['dək','tək','can','cən']:
            
#             l14.append({"label": i['label'].split('/')[0], "start_offset": i['start_offset'], "end_offset":  i['end_offset']-3})
#             l14.append({"label": i['label'].split('/')[1], "start_offset": i['end_offset']-3, "end_offset":  i['end_offset']})
                         
#         elif s7[i["start_offset"]:i["end_offset"]][-4:] in ['sana']:
          
#             l14.append({"label": i['label'].split('/')[0], "start_offset": i['start_offset'], "end_offset":  i['end_offset']-4})
#             l14.append({"label": i['label'].split('/')[1], "start_offset": i['end_offset']-4, "end_offset":  i['end_offset']})
                   
#         else:
           
#             l14.append({"label": i['label'].split('/')[0], "start_offset": i['start_offset'], "end_offset":  i['end_offset']-2})
#             l14.append({"label": i['label'].split('/')[1], "start_offset": i['end_offset']-2, "end_offset":  i['end_offset']})
          
#     else: 
#         l14.append(i)       